In [1]:
"""
Michael E. Ramsey
CSCI 5352
Date Created: 11/01/18
Last Edited: 12/3/18

This is a python script to analyze the Network of Facebook data presented in CSCI 5352
In this file, we compute features for each edge pair and a randomly sampled set of the 
non-edge pairs. 

The output of this scipt is a csv, containing several "features" of each edge and non-edge pair.
This .csv file will be used to create machine learning models for edge prediction.
"""

# Get necessary libraries
import sys
from os import listdir
from os.path import isfile, join
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import scipy
from scipy import linalg
from scipy.sparse import csr_matrix
from random import randint
import itertools

C:\Users\Mike Ramsey\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
"""
Extract all filenames for facebook100 dataset
You may have to edit the data location
"""

# Get list of filenames that contain edge information
# Had to exclude a bunch of files that I did not need
# Could have done this more efficiently
files = [f for f in listdir("../Data/facebook100txt/") if isfile(join("../Data/facebook100txt/", f)) 
         if not(f.endswith('r.txt')) if f.endswith('.txt') if not(f.endswith('readme_aaron.txt'))
         if not(f.endswith('facebook100_readme_021011.txt'))]

In [3]:
"""
Loop through all files and create a feature list.
"""
for filename in files:
    
    # Remove .txt extension
    filename = filename[:-4]
    
    """
    Extract the nodes and edges.
    """
    # Create empty list to store connections
    edge_list = []

    # Open file and read lines
    f = open("../Data/facebook100txt/" + filename + '.txt')
    f1 = f.readlines()
    f.close()

    # Loop through lines and record edge
    for x in f1:
        temp = x[:-1].replace("\t"," ")
        edge_list.append(tuple(map(int, temp.split())))
    
    """
    Extract Node Attributes
    """
    node_attr = pd.read_table("../Data/facebook100txt/" + filename + '_attr.txt')
    
    # Create node list
    node_list = list(range(1,node_attr.shape[0] + 1))
    
    # Create list of not_edges - adjust for proportion of edges that exist in the network
    total_val = int(np.round(len(edge_list)*(1+len(edge_list)/(len(node_list)-1)**2)))
    not_edges = [(randint(1, len(node_list)), randint(1, len(node_list))) for _ in range(total_val)]
    not_edges = list(set(not_edges) - set(edge_list))
    
    # Create label vectors
    y_edges = np.ones((len(edge_list),1))
    y_not_edges = np.zeros((len(not_edges),1))
    
    """
    Perform train/valid/test split
    We will use 95/2.5/2.5
    """
    edge_train, edge_test, y_train, y_test = train_test_split(edge_list, y_edges, test_size = .025, random_state = 345)
    not_edge_train, not_edge_test, y_not_train, y_not_test = train_test_split(not_edges, y_not_edges, test_size = .025, random_state = 345)

    edge_valid, edge_test, y_valid, y_test = train_test_split(edge_test, y_test, test_size = .5, random_state = 345)
    not_edge_valid, not_edge_test, y_not_valid, y_not_test = train_test_split(not_edge_test, y_not_test, test_size = .5, random_state = 345)
    
    """
    Generate the network
    """
    G=nx.Graph()
    G.add_nodes_from(node_list)
    G.add_edges_from(edge_train)

    # Compute clustering coefficient for each ndoe
    cluster_coeff = nx.clustering(G)
    
    # Construct adjacency matrices and multiples
    A = nx.adjacency_matrix(G)
    Asquared = A.dot(A)
    Acubed = Asquared.dot(A)

    # Construct row sum sum diagonal matrix
    #D = (A.todense().sum(axis = 1)).flatten()
    #D = csr_matrix(np.diag(np.array(D)[0]))
    #Dinv = np.diag(1/np.diag(D.todense()))

    # Coompute pseudo-inverse
    #L = D-A
    #Lstar = np.linalg.pinv(L.todense())

    # Compute rooted pagerank
    #RPR001 = (1-.001)*np.linalg.inv(np.identity(A.shape[0]) - .001*(Dinv.dot(A.todense())))
    #RPR01 = (1-.01)*np.linalg.inv(np.identity(A.shape[0]) - .01*(Dinv.dot(A.todense())))
    #RPR1 = (1-.1)*np.linalg.inv(np.identity(A.shape[0]) - .1*(Dinv.dot(A.todense())))
    
    """
    Concatenate lists
    """
    edge_list = edge_train + not_edge_train + edge_test + not_edge_test + edge_valid + not_edge_valid
    y = np.concatenate((y_train, y_not_train, y_test, y_not_test, y_valid, y_not_valid))
    label = ['Tr']*(len(edge_train)+len(not_edge_train)) + ['T']*(len(edge_test)+len(not_edge_test)) + ['V']*(len(edge_valid)+len(not_edge_valid))
    
    """
    Feature Construction
    """
    # Initialize numpy arrays to store features
    shortest_path = np.zeros((len(edge_list),1))
    common_neighbors = np.zeros((len(edge_list),1))
    pref_attach = np.zeros((len(edge_list),1))
    neighbor_sum = np.zeros((len(edge_list),1))
    local_cluster_sum = np.zeros((len(edge_list),1))
    local_cluster_prod = np.zeros((len(edge_list),1))
    jaccard_coeff = np.zeros((len(edge_list),1))
    adamic_adar = np.zeros((len(edge_list),1))
    same_gender = np.zeros((len(edge_list),1))
    same_status = np.zeros((len(edge_list),1))
    same_major = np.zeros((len(edge_list),1))
    same_dorm = np.zeros((len(edge_list),1))
    same_year = np.zeros((len(edge_list),1))
    sorensen = np.zeros((len(edge_list),1))
    cosine_sim = np.zeros((len(edge_list),1))
    hub_prom = np.zeros((len(edge_list),1))
    hub_depr = np.zeros((len(edge_list),1))
    lhn = np.zeros((len(edge_list),1))
    resource_all = np.zeros((len(edge_list),1))
    local_path001 = np.zeros((len(edge_list),1))
    local_path01 = np.zeros((len(edge_list),1))
    local_path1 = np.zeros((len(edge_list),1))
    #commute_time = np.zeros((len(edge_list),1))
    #cosine_sim_time = np.zeros((len(edge_list),1))
    #rooted_page001 = np.zeros((len(edge_list),1))
    #rooted_page01 = np.zeros((len(edge_list),1))
    #rooted_page1 = np.zeros((len(edge_list),1))

    # Loop through edges to extract features
    for edge in range(0,len(edge_list)):

        # Shortest path
        if nx.has_path(G, edge_list[edge][0], edge_list[edge][1]) == True:
            shortest_path[edge] = len(nx.shortest_path(G, edge_list[edge][0], edge_list[edge][1]))-1
        else:
            shortest_path[edge] = 1000

        # Common neighbors
        common_neighbors[edge] = sum(1 for i in nx.common_neighbors(G, edge_list[edge][0], edge_list[edge][1]))

        # Preferential attachment
        pref_attach[edge] = sum(1 for i in G.neighbors(edge_list[edge][0]))*sum(1 for i in G.neighbors(edge_list[edge][1]))

        # Neighbor sum
        neighbor_sum[edge] = sum(1 for i in G.neighbors(edge_list[edge][0]))+sum(1 for i in G.neighbors(edge_list[edge][1]))

        # Jaccard coefficient
        temp = nx.jaccard_coefficient(G,[edge_list[edge]])
        jaccard_coeff[edge] = list(temp)[0][2]

        # Sorensen Index
        sorensen[edge] = common_neighbors[edge]/neighbor_sum[edge]

        # Cosine Similarity
        cosine_sim[edge] = common_neighbors[edge]/np.sqrt(pref_attach[edge])

        # Hub Promoted
        hub_prom[edge] = common_neighbors[edge]/min(sum(1 for i in G.neighbors(edge_list[edge][0])), sum(1 for i in G.neighbors(edge_list[edge][1])))

        # Hub Depressed
        hub_depr[edge] = common_neighbors[edge]/max(sum(1 for i in G.neighbors(edge_list[edge][0])), sum(1 for i in G.neighbors(edge_list[edge][1])))

        # LHN
        lhn[edge] = common_neighbors[edge]/pref_attach[edge]

        # Adamic/Adar
        temp = nx.adamic_adar_index(G,[edge_list[edge]])
        try: adamic_adar[edge] = list(temp)[0][2]
        except ZeroDivisionError: adamic_adar[edge] = 1000

        # Resource Allocation
        temp = list(nx.common_neighbors(G, edge_list[edge][0], edge_list[edge][1]))
        temp = dict(G.degree(temp))
        resource_all[edge] = sum(1/i for i in list(temp.values()))

        # Clustering coefficient
        local_cluster_sum[edge] = cluster_coeff[edge_list[edge][0]] + cluster_coeff[edge_list[edge][1]]
        local_cluster_prod[edge] = cluster_coeff[edge_list[edge][0]] * cluster_coeff[edge_list[edge][1]]

        # Local Path
        local_path001[edge] = Asquared[edge_list[edge][0]-1,edge_list[edge][1]-1] + .001*Acubed[edge_list[edge][0]-1,edge_list[edge][1]-1]
        local_path01[edge] = Asquared[edge_list[edge][0]-1,edge_list[edge][1]-1] + .01*Acubed[edge_list[edge][0]-1,edge_list[edge][1]-1]
        local_path1[edge] = Asquared[edge_list[edge][0]-1,edge_list[edge][1]-1] + .1*Acubed[edge_list[edge][0]-1,edge_list[edge][1]-1]

        # Commute Time
        #temp = (Lstar[edge_list[edge][0]-1, edge_list[edge][0]-1] + Lstar[edge_list[edge][1]-1, edge_list[edge][1]-1] - 2*Lstar[edge_list[edge][0]-1, edge_list[edge][1]-1])
        #commute_time[edge] = len(edge_list)*temp

        # Cosine similarity
        #temp = np.sqrt(Lstar[edge_list[edge][0]-1, edge_list[edge][0]-1]*Lstar[edge_list[edge][1]-1, edge_list[edge][1]-1])
        #cosine_sim_time[edge] = Lstar[edge_list[edge][0]-1, edge_list[edge][1]-1] / temp

        # Rooted pagerank
        #rooted_page001[edge] = RPR001[edge_list[edge][0]-1,edge_list[edge][1]-1]
        #rooted_page01[edge] = RPR01[edge_list[edge][0]-1,edge_list[edge][1]-1]
        #rooted_page1[edge] = RPR1[edge_list[edge][0]-1,edge_list[edge][1]-1]

        ####### Meta Data Attributes

        # Same gender
        same_gender[edge] = (node_attr.loc[edge_list[edge][0]-1,'gender'] + node_attr.loc[edge_list[edge][1]-1,'gender'])%2

        # Same status
        if node_attr.loc[edge_list[edge][0]-1,'status'] == node_attr.loc[edge_list[edge][1]-1,'status']:
            same_status[edge] = 1
        else:
            same_status[edge] = 0

        # Same major
        if node_attr.loc[edge_list[edge][0]-1,'major'] == node_attr.loc[edge_list[edge][1]-1,'major']:
            same_major[edge] = 1
        else:
            same_major[edge] = 0

        # Same dorm
        if node_attr.loc[edge_list[edge][0]-1,'dorm'] == node_attr.loc[edge_list[edge][1]-1,'dorm']:
            same_dorm[edge] = 1
        else:
            same_dorm[edge] = 0

        # Same year
        if node_attr.loc[edge_list[edge][0]-1,'year'] == node_attr.loc[edge_list[edge][1]-1,'year']:
            same_year[edge] = 1
        else:
            same_year[edge] = 0

        ####### Print statement for tracking
        if edge%10000 == 0:
            print(str(np.round(edge/len(edge_list)*100,1)) + '%' , end = " ")
        
    """
    Create data frame with all of the features
    """
    # Initialize data frame to store features
    feature_df = pd.DataFrame.from_records(edge_list, columns = ['Node_1', 'Node_2'])

    # Create features for data frame
    feature_df['shortest_path'] = shortest_path
    feature_df['common_neighbors'] = common_neighbors 
    feature_df['pref_attach'] = pref_attach 
    feature_df['neighbor_sum'] = neighbor_sum 
    feature_df['sorensen'] = sorensen 
    feature_df['cosine_sim'] = cosine_sim 
    feature_df['hub_prom'] = hub_prom 
    feature_df['hub_depr'] = hub_depr 
    feature_df['lhn'] = lhn
    feature_df['adamic_adar'] = adamic_adar
    feature_df['resource_all'] = resource_all 
    feature_df['local_cluster_sum'] = local_cluster_sum
    feature_df['local_cluster_prod'] = local_cluster_prod
    feature_df['same_gender'] = same_gender
    feature_df['same_status'] = same_status
    feature_df['same_major'] = same_major
    feature_df['same_dorm'] = same_dorm
    feature_df['same_year'] = same_year
    feature_df['local_path001'] = local_path001
    feature_df['local_path01'] = local_path01
    feature_df['local_path1'] = local_path1
    #feature_df['commute_time'] = commute_time
    #feature_df['cosine_sim_time'] = cosine_sim_time
    #feature_df['rooted_page001'] = rooted_page001
    #feature_df['rooted_page01'] = rooted_page01
    #feature_df['rooted_page1'] = rooted_page1
    feature_df['edge'] = y
    feature_df['label'] = label
    
    """
    Save features to a data frame
    """
    # Feature data frame
    feature_df.to_csv(filename + '.csv')
            
    """
    Print statement for tracking
    """
    print(filename)

0.0% 1.2% 2.3% 3.5% 4.6% 5.8% 6.9% 8.1% 9.2% 10.4% 11.5% 12.7% 13.8% 15.0% 16.1% 17.3% 18.4% 19.6% 20.7% 21.9% 23.0% 24.2% 25.3% 26.5% 27.6% 28.8% 29.9% 31.1% 32.2% 33.4% 34.6% 35.7% 36.9% 38.0% 39.2% 40.3% 41.5% 42.6% 43.8% 44.9% 46.1% 47.2% 48.4% 49.5% 50.7% 51.8% 53.0% 54.1% 55.3% 56.4% 57.6% 58.7% 59.9% 61.0% 62.2% 63.3% 64.5% 65.6% 66.8% 68.0% 69.1% 70.3% 71.4% 72.6% 73.7% 74.9% 76.0% 77.2% 78.3% 79.5% 80.6% 81.8% 82.9% 84.1% 85.2% 86.4% 87.5% 88.7% 89.8% 91.0% 92.1% 93.3% 94.4% 95.6% 96.7% 97.9% 99.0% American75
0.0% 2.8% 5.6% 8.3% 11.1% 13.9% 16.7% 19.4% 22.2% 25.0% 27.8% 30.5% 33.3% 36.1% 38.9% 41.6% 44.4% 47.2% 50.0% 52.7% 55.5% 58.3% 61.1% 63.8% 66.6% 69.4% 72.2% 74.9% 77.7% 80.5% 83.3% 86.0% 88.8% 91.6% 94.4% 97.1% 99.9% Amherst41
0.0% 3.0% 6.0% 9.0% 12.0% 14.9% 17.9% 20.9% 23.9% 26.9% 29.9% 32.9% 35.9% 38.9% 41.9% 44.8% 47.8% 50.8% 53.8% 56.8% 59.8% 62.8% 65.8% 68.8% 71.7% 74.7% 77.7% 80.7% 83.7% 86.7% 89.7% 92.7% 95.7% 98.6% Bowdoin47
0.0% 1.8% 3.7% 5.5% 7.3% 9.1% 11.0% 12

C:\Users\Mike Ramsey\Anaconda3\lib\site-packages\ipykernel_launcher.py:146: RuntimeWarning: invalid value encountered in true_divide
C:\Users\Mike Ramsey\Anaconda3\lib\site-packages\ipykernel_launcher.py:149: RuntimeWarning: invalid value encountered in true_divide
C:\Users\Mike Ramsey\Anaconda3\lib\site-packages\ipykernel_launcher.py:155: RuntimeWarning: invalid value encountered in true_divide


49.2% 50.2% 51.2% 52.2% 53.2% 54.2% 55.2% 56.2% 57.2% 58.2% 59.2% 60.2% 61.2% 62.2% 63.2% 64.2% 65.2% 66.2% 67.2% 68.2% 69.2% 70.2% 71.2% 72.2% 73.2% 74.2% 75.2% 76.2% 77.2% 78.2% 79.2% 80.2% 81.3% 82.3% 83.3% 84.3% 85.3% 86.3% 87.3% 88.3% 89.3% 90.3% 91.3% 92.3% 93.3% 94.3% 95.3% 96.3% 97.3% 98.3% 99.3% Carnegie49
0.0% 1.6% 3.2% 4.9% 6.5% 8.1% 9.7% 11.4% 13.0% 14.6% 16.2% 17.9% 19.5% 21.1% 22.7% 24.4% 26.0% 27.6% 29.2% 30.8% 32.5% 34.1% 35.7% 37.3% 39.0% 40.6% 42.2% 43.8% 45.5% 47.1% 48.7% 50.3% 52.0% 53.6% 55.2% 56.8% 58.4% 60.1% 61.7% 63.3% 64.9% 66.6% 68.2% 69.8% 71.4% 73.1% 74.7% 76.3% 77.9% 79.6% 81.2% 82.8% 84.4% 86.0% 87.7% 89.3% 90.9% 92.5% 94.2% 95.8% 97.4% 99.0% Colgate88
0.0% 2.6% 5.2% 7.9% 10.5% 13.1% 15.7% 18.3% 21.0% 23.6% 26.2% 28.8% 31.4% 34.1% 36.7% 39.3% 41.9% 44.5% 47.2% 49.8% 52.4% 55.0% 57.6% 60.3% 62.9% 65.5% 68.1% 70.7% 73.4% 76.0% 78.6% 81.2% 83.8% 86.5% 89.1% 91.7% 94.3% 96.9% 99.6% Hamilton46
0.0% 4.3% 8.5% 12.8% 17.1% 21.3% 25.6% 29.9% 34.1% 38.4% 42.7% 46.9